# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix


[nltk_data] Downloading package punkt to /Users/akniels1/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akniels1/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
inspector = inspect(engine)
inspector.get_table_names()
# for table_name in inspector.get_table_names():
#     print('')
#     for column in inspector.get_columns(table_name):
#         print("Column: %s" % column['name'])
df = pd.read_sql_table('Message', engine)
df
# X = 
# Y = 

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,26381,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,26382,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,26383,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,26384,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = text.lower()
    token = word_tokenize(text)
    words = [w for w in token if w not in stopwords.words('english')]
    stemmed = [PorterStemmer().stem(w) for w in words]
    return token

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))])

# pipeline = MultiOutputClassifier(KNeighborsClassifier())

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
x = df['message']
y = df.drop(['index', 'id', 'message', 'genre', 'original'], axis=1)
# x = pipeline['messa']
# y


x_train , x_test , y_train ,y_test  = train_test_split (x, y , test_size = .33, random_state = 45)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)
# for column in y_test:
#     f1 = classification_report(y_test[column].array, y_pred[column].array)
#     print( column, f1)

In [10]:
#[row[0] for row in y_pred]
#y_test['request'].values
for index ,column in enumerate(y_test, 0):
    f1 = classification_report(y_test[column].values, [row[index] for row in y_pred])
    print( column)
    print(f1)

related
             precision    recall  f1-score   support

          0       0.51      0.53      0.52      1993
          1       0.85      0.85      0.85      6588
          2       0.71      0.21      0.33        71

avg / total       0.77      0.77      0.77      8652

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      7227
          1       0.70      0.51      0.59      1425

avg / total       0.87      0.88      0.88      8652

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      8603
          1       0.00      0.00      0.00        49

avg / total       0.99      0.99      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.71      0.78      0.75      5059
          1       0.65      0.56      0.60      3593

avg / total       0.69      0.69      0.69      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
parameters = {
        'vectorizer__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__leaf_size': (20, 30, 50, 70, 90),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8c59bbe620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.fit(x_train, y_train)
y_pred = cv.predict(x_test)

ValueError: Invalid parameter features for estimator Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]). Check the list of available parameters with `estimator.get_params().keys()`.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.